In [71]:
import pandas as pd

model = "gemma-2-9b-it-Q8_0_L"
selected_search_prompt = 2
selected_extraction_prompt = 5
selected_summary_prompt = 1

summary = pd.read_csv('./data/summarization_results.csv')
summary["summary"] = summary["summary"].apply(lambda x: "" if x == "[]" else x)

queries = pd.read_csv('./data/web_search_results.csv')
queries = queries[(queries["prompt"] == selected_search_prompt) & (queries["model"] == model)]

page_information = pd.read_csv('./data/page_information.csv')
page_information["query"] = page_information["query"].apply(lambda x: x.replace(" -cyberleninka -youtube -видео -pdf", ""))

topics = pd.read_csv('./data/extraction_results.csv')
topics = topics[(topics["prompt"] == selected_extraction_prompt) & (topics["model"] == model)]

summary_dict = dict(zip(summary["query"], summary["summary"]))
queries_dict = dict(zip(queries["num_message"], queries["queries"]))

data = topics[["Тема", "Социальная сеть", "Целевая аудитория", "Дополнительная информация", "num_message"]].copy()
data["context"] = data["num_message"].apply(lambda x: "\n".join([summary_dict[q] for q in eval(queries_dict[x])]))

data.head()

,Тема,Социальная сеть,Целевая аудитория,Дополнительная информация,num_message,context
200,Список книг для расширения кругозора и поиска ...,ВКонтакте,"Любители литературы, интересующиеся саморазвит...",Предложить список из 5-7 книг разных жанров с ...,0,"- 2015 год\n- ""Удовольствие от Х""\n- ""Голая ст..."
201,Сколько книг нужно читать в год?,Telegram,"Любой пользователь, интересующийся саморазвити...",Поделиться личным опытом чтения. Дать советы п...,1,- Прочитайте 30 книг за 30 дней.\n- Не читайте...
202,Топ-5 упражнений для утренней зарядки,Instagram,"Люди, интересующиеся здоровым образом жизни и ...",Использовать визуальный контент: видео или фот...,2,\n- Утренняя зарядка должна быть регулярной.\...
203,История успеха известного спортсмена: преодоле...,ВКонтакте,"Люди всех возрастов, интересующиеся спортом и ...","Рассказать о жизненном пути спортсмена, его пр...",3,- 1960-70 годы - пик популярности Adidas.\n- 1...
204,Как правильно обрезать плодовые деревья весной...,Telegram,Садоводы и огородники,NaN,4,Формирующая обрезка груши в первые годы жизни ...


In [72]:
from langchain_community.llms import LlamaCpp
from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


llama3_tokens = {
    'first_token': '<|begin_of_text|>',
    'start_header': '<|start_header_id|>', 
    'end_header': '<|end_header_id|>', 
    'end_message': '<|eot_id|>'
}
gemma2_tokens = {
    'first_token': '',
    'start_header': '<start_of_turn>',
    'end_header': '',
    'end_message': '<end_of_turn>'
}

def get_llm(path):

    model = LlamaCpp(
        model_path=path,
        temperature=0,
        n_ctx=8192,
        max_tokens=2000,
        top_p=1,
        repeat_penalty=1.1,
        # callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
        verbose=False,
        n_gpu_layers=33
    )
    return model

def get_chat(model_name):
    chat = ChatOllama(model=model_name, temperature=0)
    return chat

In [79]:
generation_prompt_candidate_1 = """{first_token}{start_header}system{end_header}

Ты — русскоязычный автоматический ассистент для генерации статей и постов в социальные сети. Твоя задача — синтезировать результаты веб-поиска для создания качественного контента.

Инструкции:

Используй только необходимую информацию из контекста веб-поиска.
Не придумывай факты и не добавляй вымышленные данные.
Следуй требованиям к тексту, указанным ниже.
Сохраняй цельность и логичность поста.
Требования к тексту:

Тема: «{topic}»
Социальная сеть: «{platform}»
Целевая аудитория: «{audience}»
Дополнительная информация: «{additional_information}»
Контекст веб-поиска: «{context}»
Пример формата поста:

Введение: Кратко введи в тему, привлеки внимание аудитории.
Основная часть: Представь ключевые факты и информацию, выделенные в результате веб-поиска.
Заключение: Подведи итог, предложи действия или задавай вопросы аудитории для вовлечения.
Контент:

{end_message}{start_header}assistant{end_header}\n"""

generation_prompt_candidate_2 = """{first_token}{start_header}system{end_header}

Ты — русскоязычный автоматический ассистент для генерации статей и постов в социальные сети. Твоя задача — использовать результаты веб-поиска для создания информативного и привлекательного контента.

Инструкции:

Используй только проверенную информацию из контекста веб-поиска.
Не добавляй вымышленные данные.
Создай текст, который соответствует требованиям ниже.
Убедись, что пост логичен и цельный.
Требования к тексту:

Тема: «{topic}»
Социальная сеть: «{platform}»
Целевая аудитория: «{audience}»
Дополнительная информация: «{additional_information}»
Контекст веб-поиска: «{context}»
Формат поста:

Заголовок: Привлекающий внимание и отражающий суть темы.
Введение: Краткий обзор темы.
Основная часть: Основные факты и информация из контекста веб-поиска.
Заключение: Резюме и призыв к действию.
Контент:

{end_message}{start_header}assistant{end_header}\n"""

generation_prompt_candidate_3 = """{first_token}{start_header}system{end_header}

Ты — русскоязычный автоматический ассистент для создания статей и постов в социальные сети. Твоя цель — использовать результаты веб-поиска для создания информативного и интересного контента.

Инструкции:

Бери информацию только из контекста веб-поиска.
Не добавляй вымышленные или неподтвержденные данные.
Следуй структуре и требованиям к тексту, описанным ниже и учитывай формат социальной сети.
Убедись, что текст логичен и связен.
Требования к тексту:

Тема: «{topic}»
Социальная сеть: «{platform}»
Целевая аудитория: «{audience}»
Дополнительная информация: «{additional_information}»
Контекст веб-поиска: «{context}»
Структура поста:

Заголовок: Яркий и цепляющий.
Введение: Привлечение внимания, краткий обзор темы.
Основная часть: Подробная информация и факты из контекста веб-поиска.
Заключение: Итоговые мысли и призыв к взаимодействию с аудиторией.

Особенности социальной сети:

Facebook: Используй четкие и информативные заголовки, добавь ссылки и изображения.
Instagram: Краткость. Используй хэштеги и эмодзи.
Twitter: Лаконичность, не более 280 символов. Включи хэштеги и упоминания.
LinkedIn: Профессиональный и деловой тон. Включи ссылки на источники и исследования.
Вконтакте: Вовлекающие заголовки, ориентация на активное взаимодействие с аудиторией в комментариях.
Телеграмм: Компактные и содержательные, часто используются для быстрого информирования и взаимодействия через каналы.

Контент:

{end_message}{start_header}assistant{end_header}\n"""

generation_candidates = [
    generation_prompt_candidate_1,
    generation_prompt_candidate_2,
    generation_prompt_candidate_3
]

In [83]:
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate

import pandas as pd
import tqdm
from langchain_core.exceptions import OutputParserException


def get_generation_prompt(prompt):
    generation_prompt = PromptTemplate(
        template=prompt,
        input_variables=["first_token", "start_header", "end_header", "end_message", "topic", 
                         "platform", "audience", "additional_information", "context"],
    )
    return generation_prompt

In [86]:
from langchain_core.output_parsers import StrOutputParser

llm = get_chat(model)
input_dict = gemma2_tokens.copy()

results = pd.DataFrame()

for candidate in generation_candidates:
    generation_prompt = get_generation_prompt(candidate)
    generation_chain = generation_prompt | llm | StrOutputParser()
    outputs = {}

    for num, row in tqdm.tqdm(data.reset_index().iterrows(), total=40):
        input_dict["topic"] = row["Тема"]
        input_dict["platform"] = row["Социальная сеть"]
        input_dict["audience"] = row["Целевая аудитория"]
        input_dict["additional_information"] = row["Дополнительная информация"]
        input_dict["context"] = row["context"]
        try:
            output = {"content": generation_chain.invoke(input_dict)}
        except Exception as error:
            print(error)
            output ={"content": ""}
        
        # output = {key: output.get(key, None) for key in ["queries"]}
        output["num_message"] = num
        for key in output.keys():
            outputs[key] = outputs.get(key, []) + [output[key]]

    df = pd.DataFrame.from_dict(outputs)
    df['stage'] = "generation"
    df['prompt'] = generation_candidates.index(candidate)
    results = pd.concat([results, df], axis=0)
    results.to_csv('./data/generation_results.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [04:50<00:00,  7.27s/it]


In [87]:
results

,content,num_message,stage,prompt
0,## 📚 Ищешь вдохновение и хочешь расширить сво...,0,generation,0
1,## Сколько книг нужно читать в год? 🤔 \n\nЭто...,1,generation,0
2,## ☀️ Утренняя зарядка - залог продуктивного ...,2,generation,0
3,## Когда трудности становятся трамплином к ус...,3,generation,0
4,## 🌸 Весенняя обрезка плодовых деревьев: не п...,4,generation,0
...,...,...,...,...
35,## 🚀 Как построить команду мечты? 🏆\n\nРабота...,35,generation,2
36,## 🤢 Задыхаемся от смога? Как воздух влияет ...,36,generation,2
37,## Хочешь быть харизматичным? Развивай свои к...,37,generation,2
38,"## Путешествия - это не просто отдых, а личны...",38,generation,2


In [93]:
results[results["num_message"] == 36].iloc[0].content

'##  Чистый воздух - залог здоровья! 🍃\n\nМы все хотим быть здоровыми и энергичными, но мало кто задумывается о том, как на наше здоровье влияет окружающая среда. 😮  \n\n**Оказывается, загрязнение воздуха - одна из главных угроз для нашего организма.** Выхлопные газы, промышленные выбросы и мусоросжигательные заводы выбрасывают в атмосферу вредные вещества, которые попадают в наши легкие и кровь. 😔\n\n**Что же это значит для нас?** \n\n* **Сердечно-сосудистые заболевания**,\n* **Проблемы с репродуктивной системой**,\n* **Болезни нервной системы**,\n* **Сахарный диабет**,\n* **Рак**,\n* **Заболевания легких** - вот лишь некоторые из последствий загрязнения воздуха. 😨\n\n**Но есть и хорошие новости!** 💪 Мы можем бороться с этой проблемой:\n\n* **Переход на возобновляемые источники энергии**,\n* **Развитие экологичного транспорта**,\n* **Озеленение городов**,\n* **Создание велосипедной инфраструктуры** - все это поможет нам дышать чистым воздухом. 🌳🚲\n\n**А что вы можете сделать для себя?

In [94]:
from IPython.display import display, Markdown

display(Markdown(results[results["num_message"] == 36].iloc[0].content))

##  Чистый воздух - залог здоровья! 🍃

Мы все хотим быть здоровыми и энергичными, но мало кто задумывается о том, как на наше здоровье влияет окружающая среда. 😮  

**Оказывается, загрязнение воздуха - одна из главных угроз для нашего организма.** Выхлопные газы, промышленные выбросы и мусоросжигательные заводы выбрасывают в атмосферу вредные вещества, которые попадают в наши легкие и кровь. 😔

**Что же это значит для нас?** 

* **Сердечно-сосудистые заболевания**,
* **Проблемы с репродуктивной системой**,
* **Болезни нервной системы**,
* **Сахарный диабет**,
* **Рак**,
* **Заболевания легких** - вот лишь некоторые из последствий загрязнения воздуха. 😨

**Но есть и хорошие новости!** 💪 Мы можем бороться с этой проблемой:

* **Переход на возобновляемые источники энергии**,
* **Развитие экологичного транспорта**,
* **Озеленение городов**,
* **Создание велосипедной инфраструктуры** - все это поможет нам дышать чистым воздухом. 🌳🚲

**А что вы можете сделать для себя?** 🤔

* Установите дома очиститель воздуха,
* Гуляйте на свежем воздухе вдали от промышленных зон,
* Поддерживайте экологические инициативы!


Давайте вместе сделаем мир чище и здоровее! 💚 


In [95]:
display(Markdown(results[results["num_message"] == 36].iloc[1].content))

##  🤢  Задыхаемся в городе? Как воздух влияет на наше здоровье! 🌳

Мы все знаем, что свежий воздух полезен, но знали ли вы, что загрязнение воздуха может серьезно угрожать нашему здоровью? 😔  Это не просто неприятный запах - это реальная опасность, которая приводит к болезням и даже преждевременной смерти. 

**Что же загрязняет наш воздух?** 🏭🚗💨

* Выхлопные газы от автомобилей
* Сжигание угля на промышленных предприятиях
* Мусоросжигательные заводы

Самыми опасными являются мельчайшие частицы, которые проникают глубоко в наши легкие и даже кровь!  🩸 🫁

**Влияние загрязнения воздуха на здоровье:**

* Заболевания сердечно-сосудистой системы
* Проблемы с репродуктивной системой
* Болезни нервной системы
* Сахарный диабет
* Рак
* Болезни легких

😱  Даже беременность может быть подвержена риску из-за загрязнения воздуха, повышая вероятность рождения детей с пороками развития. 👶

**Но есть и хорошие новости! 💪** Мы можем бороться с загрязнением воздуха:

* Внедрять современные фильтры на промышленных предприятиях
* Переходить на возобновляемые источники энергии
* Создавать велосипедную инфраструктуру
* Перерабатывать вторсырье
* Озеленять города
* Развивать экологичное сельское хозяйство

**А что вы можете сделать для себя?** 🏡🚶‍♀️🌳

* Установите дома кондиционер, очиститель и увлажнитель воздуха.
* Гуляйте на свежем воздухе вдали от городов.
* Поддерживайте инициативы по защите окружающей среды!


Давайте вместе сделаем воздух чище и жизнь здоровее! 💚  #здоровье #окружающаясреда #загрязнениевоздуха #экология 



In [96]:
display(Markdown(results[results["num_message"] == 36].iloc[2].content))

##  🤢  Задыхаемся от смога? Как воздух влияет на наше здоровье! 💨

Мы все хотим быть здоровыми, но мало кто задумывается о том, как на нас влияет окружающая среда. А ведь именно она – наш главный союзник или враг в борьбе за благополучие! 🌎

Сегодня поговорим о загрязнении воздуха, которое становится всё более актуальной проблемой.  🚗🏭💨 Выхлопные газы, промышленные выбросы и мусоросжигательные заводы выбрасывают в атмосферу вредные вещества, которые негативно сказываются на нашем здоровье. 🤧

**Самые опасные из них – мелкие твердые частицы (сажа, пыль), которые проникают в кровь и нарушают работу сосудов.**  🩸 Это приводит к:

* Заболеваниям сердца и сосудов
* Проблемам с репродуктивной системой
* Болезням нервной системы
* Сахарному диабету
* Рак
* Хроническим заболеваниям легких

😱 А это ещё не всё! 

**Загрязнение воздуха даже повышает вероятность рождения детей с пороками развития.**  👶

Но есть и хорошие новости! 💪 Мы можем бороться с этой проблемой:

* Внедрять современные фильтры на промышленных предприятиях
* Переходить на возобновляемые источники энергии ☀️
* Создавать велосипедную инфраструктуру 🚲
* Перерабатывать вторсырье ♻️
* Озеленять города🌳
* Развивать экологичное сельское хозяйство 🌾

**А что вы можете сделать для себя?** 🤔

* Установите дома кондиционер, очиститель, увлажнитель и ионизатор воздуха.
* Гуляйте на свежем воздухе вдали от городов.
* Поддерживайте инициативы по улучшению экологической обстановки! 🤝


**Давайте вместе сделаем мир чище и здоровее!  🌎❤️**

#здоровье #окружающаясреда #загрязнениевоздуха #экология #чистыйвоздух #смог #берегисебя #будьтездоровы





In [129]:
import nest_asyncio
nest_asyncio.apply()

In [159]:
with open("../data/user_messages.txt", "r", encoding='UTF-8') as file:
    messages = [line.rstrip() for line in file]

test_dataset = pd.DataFrame()
test_dataset["question"] = messages * 3
test_dataset["contexts"] = test_dataset["question"].apply(lambda x: [summary_dict[q] for q in eval(queries_dict[messages.index(x)])])
test_dataset["answer"] = results["content"].tolist()
test_dataset["ground_truth"] = test_dataset["answer"].copy()

In [165]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
from langchain_community.embeddings import OllamaEmbeddings
from datasets import Dataset

metrics = pd.DataFrame(
    columns=[
        "context_precision",
        "faithfulness",
        "answer_relevancy",
        "context_recall"
    ])

critic_model = "gemma-2-27b-it-Q6_K_L"
langchain_llm = ChatOllama(model=critic_model, temperature=0)
langchain_embeddings = OllamaEmbeddings(model=critic_model)

for idx, sample in test_dataset.iterrows():

    r = evaluate(
        Dataset.from_pandas(pd.DataFrame(sample).T[['question', 'ground_truth', 'answer', 'contexts']]),
        metrics=[context_precision,
                    faithfulness,
                    answer_relevancy,
                    context_recall],
        llm=langchain_llm,
        embeddings=langchain_embeddings,
        raise_exceptions=False
    )

    r = dict(r)
    metrics = pd.concat([metrics, pd.DataFrame([r])], axis=0)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\Khoreks\AppData\Local\Temp\ipykernel_25680\3813381523.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics = pd.concat([metrics, pd.DataFrame([r])], axis=0)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

...

Failed to parse output. Returning None.
D:\programs\miniconda3\envs\llm_eval\Lib\site-packages\ragas\evaluation.py:304: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Failed to parse output. Returning None.
D:\programs\miniconda3\envs\llm_eval\Lib\site-packages\ragas\evaluation.py:304: RuntimeWarning: Mean of empty slice
  value = np.nanmean(self.scores[cn])


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [167]:
metrics.to_csv('./total_metrics.csv', index=False)

In [178]:
pd.concat([metrics.reset_index(drop=True), results[["prompt"]].reset_index(drop=True)], axis=1).set_index(['prompt']).groupby(level=['prompt'])[
    ["context_precision", "faithfulness", "answer_relevancy", "context_recall"]].mean()

,context_precision,faithfulness,answer_relevancy,context_recall
prompt,,,,
0,0.679487,0.808976,0.142682,0.626406
1,0.723684,0.666081,0.140762,0.568128
2,0.710526,0.779070,0.139452,0.588633


In [180]:
pd.concat([metrics.reset_index(drop=True), results[["prompt"]].reset_index(drop=True)], axis=1).set_index(['prompt']).groupby(level=['prompt'])[
    ["context_precision", "faithfulness", "answer_relevancy", "context_recall"]].count()

,context_precision,faithfulness,answer_relevancy,context_recall
prompt,,,,
0,39,27,39,35
1,38,22,39,33
2,38,27,39,34


1. Меньше копипасты из контекста, самый выоский показатель верных утверждений.  
2. Много ошибок в работе на стадии тестирования.  
3. Заимствует больше текста из контекста из за чего происходит просадка по релевантности ответа.  